# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

The code in this notebook was developed using ChatGpt 4.5

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Essential Imports
!pip install scikit-learn

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification


Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load Dataset
dataset = load_dataset("imdb", split={'train': 'train[:500]', 'test': 'test[:500]'})
metric = load_metric("accuracy", trust_remote_code=True)

# Load Pretrained Model and Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/tmp/ipykernel_306/703326685.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy", trust_remote_code=True)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Preprocess Dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [4]:
# Initial Evaluation Metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
# Initial Model Evaluation
training_args_eval = TrainingArguments(output_dir="test_eval", per_device_eval_batch_size=16)
trainer_eval = Trainer(model=model, args=training_args_eval, eval_dataset=tokenized_datasets['test'], compute_metrics=compute_metrics)
print("Pre-trained Model Performance:", trainer_eval.evaluate())

Pre-trained Model Performance: {'eval_loss': 0.689146876335144, 'eval_accuracy': 0.64, 'eval_runtime': 2.6809, 'eval_samples_per_second': 186.504, 'eval_steps_per_second': 11.936}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:
# Configure and Apply LoRA PEFT
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [7]:
# Training Setup
training_args = TrainingArguments(
    output_dir="./lora_imdb_results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

In [8]:
# Train PEFT Model
trainer.train()

# Save PEFT Model
peft_model.save_pretrained("./lora_imdb_model")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.003392,1.000000


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [9]:
# Load and Evaluate PEFT Model for Inference

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model_loaded = AutoPeftModelForSequenceClassification.from_pretrained("./lora_imdb_model").to(device)

# Update Trainer for Inference
trainer_eval.model = peft_model_loaded

# Evaluate Fine-tuned Model
print("Fine-tuned Model Performance:", trainer_eval.evaluate())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned Model Performance: {'eval_loss': 0.0033916402608156204, 'eval_accuracy': 1.0, 'eval_runtime': 2.1464, 'eval_samples_per_second': 232.948, 'eval_steps_per_second': 14.909}
